## Greating EnzymeML document for PPAT:

In [41]:
import numpy as np
import pandas as pd
import glob, pickle, os
import numpy as np

from pyenzyme import (
    EnzymeMLDocument,
    Reactant, Complex, Vessel,
    Protein, EnzymeReaction,
    KineticModel, Measurement,
    Replicate
)

## Initial Set-up of EnzymeML document. Adding of vessels, proteins and reactants:

In [42]:
# Start of the EnzymeMLDocument
enzmldoc = EnzymeMLDocument(name="Phosphopantetheine adenylyltransferase (PPAT)")

# Adding Vessel:
vessel = Vessel(name="96 well plate", volume=150, unit="ul")
vessel_id = enzmldoc.addVessel(vessel)

# Adding protein and reactants:
PpanSH_id = enzmldoc.addReactant(Reactant(
    name = "4'-Phosphopantetheine", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = 'O=C(NCCS)CCNC(=O)[C@H](O)C(C)(C)COP(=O)(O)O',
    chebi_id = 'CHEBI:4222',
))

ATP_id = enzmldoc.addReactant(Reactant(
    name = "Adenosine triphosphate", vessel_id = vessel_id, init_conc = 3,
    unit = "mM", constant = False, smiles = 'Nc1ncnc2n(cnc12)[COOH]1O[COH](COP(O)(=O)OP(O)(=O)OP(O)(O)=O)[COOH](O)[COH]1O',
    chebi_id = 'CHEBI:15422',
))

PPi_id = enzmldoc.addReactant(Reactant(
    name = "Inorganic Pyrophospahte", vessel_id = vessel_id, init_conc = 1,
    unit = "mM", constant = False, smiles = '[O-]P(=O)([O-])OP(=O)([O-])[O-]',
    chebi_id = 'CHEBI:18361',
))

In [43]:
PPAT_id = enzmldoc.addProtein(Protein(
    name="PPAT", sequence = "UNKNOWN", vessel_id=vessel_id, 
    organism="Staphylococcus aureus", init_conc=0.1, unit="mg / ml",
))

## Adding the reaction to the EnzymeML document: 

In [44]:
# Creating the reaction:

reaction1 = EnzymeReaction(temperature=25.0, temperature_unit="C", 
                           ph=7.8, name="Phosphopantetheine adenylyltransferase", reversible=False)

reaction1.addEduct(PpanSH_id, 1.0, enzmldoc)
reaction1.addEduct(ATP_id, 1.0, enzmldoc)
reaction1.addProduct(PPi_id, 1.0, enzmldoc)
reaction1.addModifier(PPAT_id, 1.0, enzmldoc, False)



In [45]:
# Add the reactions to the  EnzymeML document

enzmldoc.addReactions([reaction1])

{'Phosphopantetheine adenylyltransferase': 'r0'}

## Adding  measurements 

(1) Create measurment for first condition at ATP = 3mM and PpanSH = 1mM

In [15]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_3 mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_3 mM.xlsx")

# Create the Measurement object for one run at ATP = 3 mM
measurement_1 = Measurement(
    name="Run 1 - ATP = 3mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_1.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_1.addData(reactant_id=ATP_id, init_conc=3, unit="mM")
measurement_1.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 3 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 3 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A03'].tolist()
)



# Add replicates to measurement
measurement_1.addReplicates(repeat1, enzmldoc)
measurement_1.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_1_id = enzmldoc.addMeasurement(measurement_1)

print("Measurement created for two repeats at ATP = 3 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 3 mM, PpanSH = 1 mM 


(2) Create measurment for the condition at ATP = 3 mM and PpanSH = 0.5 mM

In [16]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_3 mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_3 mM.xlsx")

# Create the Measurement object for one run at ATP = 3 mM
measurement_2 = Measurement(
    name="Run 2 - ATP = 3mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_2.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_2.addData(reactant_id=ATP_id, init_conc=3, unit="mM")
measurement_2.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 3 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 3 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B03'].tolist()
)



# Add replicates to measurement
measurement_2.addReplicates(repeat1, enzmldoc)
measurement_2.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_2_id = enzmldoc.addMeasurement(measurement_2)

print("Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.5 mM 


(3) Create measurment for the condition at ATP = 3 mM and PpanSH = 0.25 mM

In [17]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_3 mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_3 mM.xlsx")

# Create the Measurement object for one run at ATP = 3 mM
measurement_3 = Measurement(
    name="Run 3 - ATP = 3mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_3.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_3.addData(reactant_id=ATP_id, init_conc=3, unit="mM")
measurement_3.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 3 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 3 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C03'].tolist()
)



# Add replicates to measurement
measurement_3.addReplicates(repeat1, enzmldoc)
measurement_3.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_3_id = enzmldoc.addMeasurement(measurement_3)

print("Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.25 mM 


(4) Create measurment for the condition at ATP = 3 mM and PpanSH = 0.125 mM

In [18]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_3 mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_3 mM.xlsx")

# Create the Measurement object for one run at ATP = 3 mM
measurement_4 = Measurement(
    name="Run 4 - ATP = 3mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_4.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_4.addData(reactant_id=ATP_id, init_conc=3, unit="mM")
measurement_4.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 3 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 3 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D03'].tolist()
)



# Add replicates to measurement
measurement_4.addReplicates(repeat1, enzmldoc)
measurement_4.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_4_id = enzmldoc.addMeasurement(measurement_4)

print("Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.125 mM 


(5) Create measurment for the condition at ATP = 3 mM and PpanSH = 0.0625 mM

In [19]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_3 mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_3 mM.xlsx")

# Create the Measurement object for one run at ATP = 3 mM
measurement_5 = Measurement(
    name="Run 5 - ATP = 3mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_5.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_5.addData(reactant_id=ATP_id, init_conc=3, unit="mM")
measurement_5.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 3 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 3 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E03'].tolist()
)



# Add replicates to measurement
measurement_5.addReplicates(repeat1, enzmldoc)
measurement_5.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_5_id = enzmldoc.addMeasurement(measurement_5)

print("Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.0625 mM 


(6) Create measurment for the condition at ATP = 3 mM and PpanSH = 0.0312 mM

In [20]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_3 mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_3 mM.xlsx")

# Create the Measurement object for one run at ATP = 3 mM
measurement_6 = Measurement(
    name="Run 6 - ATP = 3mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_6.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_6.addData(reactant_id=ATP_id, init_conc=3, unit="mM")
measurement_6.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 3 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 3 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F03'].tolist()
)



# Add replicates to measurement
measurement_6.addReplicates(repeat1, enzmldoc)
measurement_6.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_6_id = enzmldoc.addMeasurement(measurement_6)

print("Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 3 mM, PpanSH = 0.0312 mM 


(7) Create measurment for first condition at ATP = 1mM and PpanSH = 1mM

In [7]:
# Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_1mM.xlsx")

# Create the Measurement object for one run at ATP = 1 mM
measurement_7 = Measurement(
    name="Run 7 - ATP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_7.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_7.addData(reactant_id=ATP_id, init_conc=1, unit="mM")
measurement_7.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 1 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A07'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 1 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A05'].tolist()
)



# Add replicates to measurement
measurement_7.addReplicates(repeat1, enzmldoc)
measurement_7.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_7_id = enzmldoc.addMeasurement(measurement_7)

print("Measurement created for two repeats at ATP = 1 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 1 mM, PpanSH = 1 mM 


(8) Create measurment for condition at ATP = 1mM and PpanSH = 0.5mM

In [9]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_1mM.xlsx")

# Create the Measurement object for one run at ATP = 1 mM
measurement_8 = Measurement(
    name="Run 8 - ATP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_8.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_8.addData(reactant_id=ATP_id, init_conc=1, unit="mM")
measurement_8.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 1 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B07'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 1 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B05'].tolist()
)



# Add replicates to measurement
measurement_8.addReplicates(repeat1, enzmldoc)
measurement_8.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_8_id = enzmldoc.addMeasurement(measurement_8)

print("Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.5 mM 


(9) Create measurment for condition at ATP = 1mM and PpanSH = 0.25mM

In [10]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_1mM.xlsx")

# Create the Measurement object for one run at ATP = 1 mM
measurement_9 = Measurement(
    name="Run 9 - ATP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_9.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_9.addData(reactant_id=ATP_id, init_conc=1, unit="mM")
measurement_9.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 1 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C07'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 1 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C05'].tolist()
)



# Add replicates to measurement
measurement_9.addReplicates(repeat1, enzmldoc)
measurement_9.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_9_id = enzmldoc.addMeasurement(measurement_9)

print("Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.25 mM 


(10) Create measurment for condition at ATP = 1mM and PpanSH = 0.125mM

In [11]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_1mM.xlsx")

# Create the Measurement object for one run at ATP = 1 mM
measurement_10 = Measurement(
    name="Run 10 - ATP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_10.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_10.addData(reactant_id=ATP_id, init_conc=1, unit="mM")
measurement_10.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 1 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D07'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 1 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D05'].tolist()
)



# Add replicates to measurement
measurement_10.addReplicates(repeat1, enzmldoc)
measurement_10.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_10_id = enzmldoc.addMeasurement(measurement_10)

print("Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.125 mM 


(11) Create measurment for condition at ATP = 1mM and PpanSH = 0.0625mM

In [12]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_1mM.xlsx")

# Create the Measurement object for one run at ATP = 1 mM
measurement_11 = Measurement(
    name="Run 11 - ATP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_11.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_11.addData(reactant_id=ATP_id, init_conc=1, unit="mM")
measurement_11.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 1 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E07'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 1 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E05'].tolist()
)



# Add replicates to measurement
measurement_11.addReplicates(repeat1, enzmldoc)
measurement_11.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_11_id = enzmldoc.addMeasurement(measurement_11)

print("Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.0625 mM 


(12) Create measurment for condition at ATP = 1mM and PpanSH = 0.0312mM

In [13]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_1mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_1mM.xlsx")

# Create the Measurement object for one run at ATP = 1 mM
measurement_12 = Measurement(
    name="Run 12 - ATP = 1mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_12.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_12.addData(reactant_id=ATP_id, init_conc=1, unit="mM")
measurement_12.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 1 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F07'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 1 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F05'].tolist()
)



# Add replicates to measurement
measurement_12.addReplicates(repeat1, enzmldoc)
measurement_12.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_12_id = enzmldoc.addMeasurement(measurement_12)

print("Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 1 mM, PpanSH = 0.0312 mM 


(13) Create measurment for first condition at ATP = 0.5mM and PpanSH = 1mM

In [14]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_5.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_5.xlsx")

# Create the Measurement object for one run at ATP = 0.5 mM
measurement_13 = Measurement(
    name="Run 13 - ATP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_13.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_13.addData(reactant_id=ATP_id, init_conc=0.5, unit="mM")
measurement_13.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.5 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A09'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.5 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A11'].tolist()
)



# Add replicates to measurement
measurement_13.addReplicates(repeat1, enzmldoc)
measurement_13.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_13_id = enzmldoc.addMeasurement(measurement_13)

print("Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 1 mM 


(14) Create measurment for first condition at ATP = 0.5mM and PpanSH = 0.5mM

In [15]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_5.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_5.xlsx")

# Create the Measurement object for one run at ATP = 0.5 mM
measurement_14 = Measurement(
    name="Run 14 - ATP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_14.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_14.addData(reactant_id=ATP_id, init_conc=0.5, unit="mM")
measurement_14.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.5 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B09'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.5 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B11'].tolist()
)



# Add replicates to measurement
measurement_14.addReplicates(repeat1, enzmldoc)
measurement_14.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_14_id = enzmldoc.addMeasurement(measurement_14)

print("Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.5 mM 


(15) Create measurment for first condition at ATP = 0.5mM and PpanSH = 0.25mM

In [16]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_5.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_5.xlsx")

# Create the Measurement object for one run at ATP = 0.5 mM
measurement_15 = Measurement(
    name="Run 15 - ATP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_15.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_15.addData(reactant_id=ATP_id, init_conc=0.5, unit="mM")
measurement_15.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.5 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C09'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.5 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C11'].tolist()
)



# Add replicates to measurement
measurement_15.addReplicates(repeat1, enzmldoc)
measurement_15.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_15_id = enzmldoc.addMeasurement(measurement_15)

print("Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.25 mM 


(16) Create measurment for first condition at ATP = 0.5mM and PpanSH = 0.125mM

In [17]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_5.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_5.xlsx")

# Create the Measurement object for one run at ATP = 0.5 mM
measurement_16 = Measurement(
    name="Run 16 - ATP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_16.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_16.addData(reactant_id=ATP_id, init_conc=0.5, unit="mM")
measurement_16.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.5 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D09'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.5 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D11'].tolist()
)



# Add replicates to measurement
measurement_16.addReplicates(repeat1, enzmldoc)
measurement_16.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_16_id = enzmldoc.addMeasurement(measurement_16)

print("Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.125 mM 


(17) Create measurment for first condition at ATP = 0.5mM and PpanSH = 0.0625mM

In [18]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_5.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_5.xlsx")

# Create the Measurement object for one run at ATP = 0.5 mM
measurement_17 = Measurement(
    name="Run 17 - ATP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_17.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_17.addData(reactant_id=ATP_id, init_conc=0.5, unit="mM")
measurement_17.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.5 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E09'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.5 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E11'].tolist()
)



# Add replicates to measurement
measurement_17.addReplicates(repeat1, enzmldoc)
measurement_17.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_17_id = enzmldoc.addMeasurement(measurement_17)

print("Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.0625 mM 


(18) Create measurment for first condition at ATP = 0.5mM and PpanSH = 0.0312mM

In [19]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_5.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_5.xlsx")

# Create the Measurement object for one run at ATP = 0.5 mM
measurement_18 = Measurement(
    name="Run 18 - ATP = 0.5mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_18.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_18.addData(reactant_id=ATP_id, init_conc=0.5, unit="mM")
measurement_18.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.5 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F09'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.5 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F11'].tolist()
)



# Add replicates to measurement
measurement_18.addReplicates(repeat1, enzmldoc)
measurement_18.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_18_id = enzmldoc.addMeasurement(measurement_18)

print("Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 0.5 mM, PpanSH = 0.0312 mM 


(19) Create measurment for first condition at ATP = 0.25mM and PpanSH = 1mM

In [20]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_25mM.xlsx")

# Create the Measurement object for one run at ATP = 0.25 mM
measurement_19 = Measurement(
    name="Run 19 - ATP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_19.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_19.addData(reactant_id=ATP_id, init_conc=0.25, unit="mM")
measurement_19.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.25 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.25 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A01'].tolist()
)



# Add replicates to measurement
measurement_19.addReplicates(repeat1, enzmldoc)
measurement_19.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_19_id = enzmldoc.addMeasurement(measurement_19)

print("Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 1 mM 


(20) Create measurment for condition at ATP = 0.25mM and PpanSH = 0.5mM

In [21]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_25mM.xlsx")

# Create the Measurement object for one run at ATP = 0.25 mM
measurement_20 = Measurement(
    name="Run 20 - ATP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_20.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_20.addData(reactant_id=ATP_id, init_conc=0.25, unit="mM")
measurement_20.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.25 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.25 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B01'].tolist()
)



# Add replicates to measurement
measurement_20.addReplicates(repeat1, enzmldoc)
measurement_20.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_20_id = enzmldoc.addMeasurement(measurement_20)

print("Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.5 mM 


(21) Create measurment for condition at ATP = 0.25mM and PpanSH = 0.25mM

In [22]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_25mM.xlsx")

# Create the Measurement object for one run at ATP = 0.25 mM
measurement_21 = Measurement(
    name="Run 21 - ATP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_21.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_21.addData(reactant_id=ATP_id, init_conc=0.25, unit="mM")
measurement_21.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.25 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.25 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C01'].tolist()
)



# Add replicates to measurement
measurement_21.addReplicates(repeat1, enzmldoc)
measurement_21.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_21_id = enzmldoc.addMeasurement(measurement_21)

print("Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.25 mM 


(22) Create measurment for condition at ATP = 0.25mM and PpanSH = 0.125mM

In [23]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_25mM.xlsx")

# Create the Measurement object for one run at ATP = 0.25 mM
measurement_22 = Measurement(
    name="Run 22 - ATP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_22.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_22.addData(reactant_id=ATP_id, init_conc=0.25, unit="mM")
measurement_22.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.25 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.25 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D01'].tolist()
)



# Add replicates to measurement
measurement_22.addReplicates(repeat1, enzmldoc)
measurement_22.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_22_id = enzmldoc.addMeasurement(measurement_22)

print("Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.125 mM 


(23) Create measurment for condition at ATP = 0.25mM and PpanSH = 0.0625mM

In [24]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_25mM.xlsx")

# Create the Measurement object for one run at ATP = 0.25 mM
measurement_23 = Measurement(
    name="Run 23 - ATP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_23.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_23.addData(reactant_id=ATP_id, init_conc=0.25, unit="mM")
measurement_23.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.25 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.25 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E01'].tolist()
)



# Add replicates to measurement
measurement_23.addReplicates(repeat1, enzmldoc)
measurement_23.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_23_id = enzmldoc.addMeasurement(measurement_23)

print("Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.0625 mM 


(24) Create measurment for condition at ATP = 0.25mM and PpanSH = 0.0312mM

In [25]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_25mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_25mM.xlsx")

# Create the Measurement object for one run at ATP = 0.25 mM
measurement_24 = Measurement(
    name="Run 24 - ATP = 0.25mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_24.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_24.addData(reactant_id=ATP_id, init_conc=0.25, unit="mM")
measurement_24.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.25 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.25 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F01'].tolist()
)



# Add replicates to measurement
measurement_24.addReplicates(repeat1, enzmldoc)
measurement_24.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_24_id = enzmldoc.addMeasurement(measurement_24)

print("Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 0.25 mM, PpanSH = 0.0312 mM 


(25) Create measurment for first condition at ATP = 0.125mM and PpanSH = 1mM

In [26]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_125mM.xlsx")

# Create the Measurement object for one run at ATP = 0.125 mM
measurement_25 = Measurement(
    name="Run 25 - ATP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_25.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_25.addData(reactant_id=ATP_id, init_conc=0.125, unit="mM")
measurement_25.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.125 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A03'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.125 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A06'].tolist()
)



# Add replicates to measurement
measurement_25.addReplicates(repeat1, enzmldoc)
measurement_25.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_25_id = enzmldoc.addMeasurement(measurement_25)

print("Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 1 mM 


(26) Create measurment for condition at ATP = 0.125mM and PpanSH = 0.5mM

In [27]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_125mM.xlsx")

# Create the Measurement object for one run at ATP = 0.125 mM
measurement_26 = Measurement(
    name="Run 26 - ATP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_26.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_26.addData(reactant_id=ATP_id, init_conc=0.125, unit="mM")
measurement_26.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.125 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B03'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.125 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B06'].tolist()
)



# Add replicates to measurement
measurement_26.addReplicates(repeat1, enzmldoc)
measurement_26.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_26_id = enzmldoc.addMeasurement(measurement_26)

print("Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.5 mM 


(27) Create measurment for condition at ATP = 0.125mM and PpanSH = 0.25mM

In [28]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_125mM.xlsx")

# Create the Measurement object for one run at ATP = 0.125 mM
measurement_27 = Measurement(
    name="Run 27 - ATP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_27.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_27.addData(reactant_id=ATP_id, init_conc=0.125, unit="mM")
measurement_27.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.125 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C03'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.125 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C06'].tolist()
)



# Add replicates to measurement
measurement_27.addReplicates(repeat1, enzmldoc)
measurement_27.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_27_id = enzmldoc.addMeasurement(measurement_27)

print("Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.25 mM 


(28) Create measurment for condition at ATP = 0.125mM and PpanSH = 0.125mM

In [29]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_125mM.xlsx")

# Create the Measurement object for one run at ATP = 0.125 mM
measurement_28 = Measurement(
    name="Run 28 - ATP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_28.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_28.addData(reactant_id=ATP_id, init_conc=0.125, unit="mM")
measurement_28.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.125 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D03'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.125 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D06'].tolist()
)



# Add replicates to measurement
measurement_28.addReplicates(repeat1, enzmldoc)
measurement_28.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_28_id = enzmldoc.addMeasurement(measurement_28)

print("Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.125 mM 


(29) Create measurment for condition at ATP = 0.125mM and PpanSH = 0.0625mM

In [30]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_125mM.xlsx")

# Create the Measurement object for one run at ATP = 0.125 mM
measurement_29 = Measurement(
    name="Run 29 - ATP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_29.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_29.addData(reactant_id=ATP_id, init_conc=0.125, unit="mM")
measurement_29.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.125 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E03'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.125 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E06'].tolist()
)



# Add replicates to measurement
measurement_29.addReplicates(repeat1, enzmldoc)
measurement_29.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_29_id = enzmldoc.addMeasurement(measurement_29)

print("Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.0625 mM 


(30) Create measurment for condition at ATP = 0.125mM and PpanSH = 0.0312mM

In [31]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_125mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_125mM.xlsx")

# Create the Measurement object for one run at ATP = 0.125 mM
measurement_30 = Measurement(
    name="Run 30 - ATP = 0.125mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_30.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_30.addData(reactant_id=ATP_id, init_conc=0.125, unit="mM")
measurement_30.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.125 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F03'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.125 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F06'].tolist()
)



# Add replicates to measurement
measurement_30.addReplicates(repeat1, enzmldoc)
measurement_30.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_30_id = enzmldoc.addMeasurement(measurement_30)

print("Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 0.125 mM, PpanSH = 0.0312 mM 


(31) Create measurment for first condition at ATP = 0.0625mM and PpanSH = 1mM

In [34]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_0625mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_0625mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0625 mM
measurement_31 = Measurement(
    name="Run 31 - ATP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_31.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_31.addData(reactant_id=ATP_id, init_conc=0.0625, unit="mM")
measurement_31.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.0625 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A08'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0625 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A10'].tolist()
)



# Add replicates to measurement
measurement_31.addReplicates(repeat1, enzmldoc)
measurement_31.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_31_id = enzmldoc.addMeasurement(measurement_31)

print("Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 1 mM 


(32) Create measurment for condition at ATP = 0.0625mM and PpanSH = 0.5mM

In [35]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_0625mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_0625mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0625 mM
measurement_32 = Measurement(
    name="Run 32 - ATP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_32.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_32.addData(reactant_id=ATP_id, init_conc=0.0625, unit="mM")
measurement_32.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.0625 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B08'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0625 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B10'].tolist()
)



# Add replicates to measurement
measurement_32.addReplicates(repeat1, enzmldoc)
measurement_32.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_32_id = enzmldoc.addMeasurement(measurement_32)

print("Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.5 mM 


(33) Create measurment for condition at ATP = 0.0625mM and PpanSH = 0.25mM

In [36]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_0625mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_0625mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0625 mM
measurement_33 = Measurement(
    name="Run 33 - ATP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_33.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_33.addData(reactant_id=ATP_id, init_conc=0.0625, unit="mM")
measurement_33.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.0625 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C08'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0625 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C10'].tolist()
)



# Add replicates to measurement
measurement_33.addReplicates(repeat1, enzmldoc)
measurement_33.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_33_id = enzmldoc.addMeasurement(measurement_33)

print("Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.25 mM 


(34) Create measurment for condition at ATP = 0.0625mM and PpanSH = 0.125mM

In [37]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_0625mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_0625mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0625 mM
measurement_34 = Measurement(
    name="Run 34 - ATP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_34.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_34.addData(reactant_id=ATP_id, init_conc=0.0625, unit="mM")
measurement_34.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.0625 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D08'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0625 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D10'].tolist()
)



# Add replicates to measurement
measurement_34.addReplicates(repeat1, enzmldoc)
measurement_34.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_34_id = enzmldoc.addMeasurement(measurement_34)

print("Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.125 mM 


(35) Create measurment for condition at ATP = 0.0625mM and PpanSH = 0.0625mM

In [38]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_0625mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_0625mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0625 mM
measurement_35 = Measurement(
    name="Run 35 - ATP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_35.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_35.addData(reactant_id=ATP_id, init_conc=0.0625, unit="mM")
measurement_35.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.0625 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E08'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0625 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E10'].tolist()
)



# Add replicates to measurement
measurement_35.addReplicates(repeat1, enzmldoc)
measurement_35.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_35_id = enzmldoc.addMeasurement(measurement_35)

print("Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.0625 mM 


(36) Create measurment for condition at ATP = 0.0625mM and PpanSH = 0.0312mM

In [39]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_0625mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_0625mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0625 mM
measurement_36 = Measurement(
    name="Run 36 - ATP = 0.0625mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_36.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_36.addData(reactant_id=ATP_id, init_conc=0.0625, unit="mM")
measurement_36.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP = 0.0625 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F08'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0625 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F10'].tolist()
)



# Add replicates to measurement
measurement_36.addReplicates(repeat1, enzmldoc)
measurement_36.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_36_id = enzmldoc.addMeasurement(measurement_36)

print("Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 0.0625 mM, PpanSH = 0.0312 mM 


(37) Create measurment for first condition at ATP = 0.0312mM and PpanSH = 1mM

In [51]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_031mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_031mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0312 mM
measurement_37 = Measurement(
    name="Run 37 - ATP = 0.0312mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_37.addData(reactant_id=PpanSH_id, init_conc=1, unit="mM")
measurement_37.addData(reactant_id=ATP_id, init_conc=0.0312, unit="mM")
measurement_37.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP =0.0312 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['A01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0312 mM @ 1 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['A03'].tolist()
)



# Add replicates to measurement
measurement_37.addReplicates(repeat1, enzmldoc)
measurement_37.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_37_id = enzmldoc.addMeasurement(measurement_37)

print("Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 1 mM ")

Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 1 mM 


(38) Create measurment for condition at ATP = 0.0312mM and PpanSH = 0.5mM

In [52]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_031mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_031mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0312 mM
measurement_38 = Measurement(
    name="Run 38 - ATP = 0.0312mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_38.addData(reactant_id=PpanSH_id, init_conc=0.5, unit="mM")
measurement_38.addData(reactant_id=ATP_id, init_conc=0.0312, unit="mM")
measurement_38.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP =0.0312 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['B01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0312 mM @ 0.5 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['B03'].tolist()
)



# Add replicates to measurement
measurement_38.addReplicates(repeat1, enzmldoc)
measurement_38.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_38_id = enzmldoc.addMeasurement(measurement_38)

print("Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.5 mM ")

Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.5 mM 


(39) Create measurment for condition at ATP = 0.0312mM and PpanSH = 0.25mM

In [53]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_031mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_031mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0312 mM
measurement_39 = Measurement(
    name="Run 39 - ATP = 0.0312mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_39.addData(reactant_id=PpanSH_id, init_conc=0.25, unit="mM")
measurement_39.addData(reactant_id=ATP_id, init_conc=0.0312, unit="mM")
measurement_39.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP =0.0312 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['C01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0312 mM @ 0.25 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['C03'].tolist()
)



# Add replicates to measurement
measurement_39.addReplicates(repeat1, enzmldoc)
measurement_39.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_39_id = enzmldoc.addMeasurement(measurement_39)

print("Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.25 mM ")

Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.25 mM 


(40) Create measurment for condition at ATP = 0.0312mM and PpanSH = 0.125mM

In [54]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_031mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_031mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0312 mM
measurement_40 = Measurement(
    name="Run 40 - ATP = 0.0312mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_40.addData(reactant_id=PpanSH_id, init_conc=0.125, unit="mM")
measurement_40.addData(reactant_id=ATP_id, init_conc=0.0312, unit="mM")
measurement_40.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP =0.0312 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['D01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0312 mM @ 0.125 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['D03'].tolist()
)



# Add replicates to measurement
measurement_40.addReplicates(repeat1, enzmldoc)
measurement_40.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_40_id = enzmldoc.addMeasurement(measurement_40)

print("Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.125 mM ")

Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.125 mM 


(41) Create measurment for condition at ATP = 0.0312mM and PpanSH = 0.0625mM

In [55]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_031mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_031mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0312 mM
measurement_41 = Measurement(
    name="Run 41 - ATP = 0.0312mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_41.addData(reactant_id=PpanSH_id, init_conc=0.0625, unit="mM")
measurement_41.addData(reactant_id=ATP_id, init_conc=0.0312, unit="mM")
measurement_41.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP =0.0312 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['E01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0312 mM @ 0.0625 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['E03'].tolist()
)



# Add replicates to measurement
measurement_41.addReplicates(repeat1, enzmldoc)
measurement_41.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_41_id = enzmldoc.addMeasurement(measurement_41)

print("Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.0625 mM ")

Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.0625 mM 


(42) Create measurment for condition at ATP = 0.0312mM and PpanSH = 0.0312mM

In [56]:
#Load your Excel file and read the sheets for each repeat
df1 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run1-ATP_0_031mM.xlsx")
df2 = pd.read_excel(r"C:\Users\Francel\Masters project\Pyenzyme\PPAT\Run2-ATP_0_031mM.xlsx")

# Create the Measurement object for one run at ATP = 0.0312 mM
measurement_42 = Measurement(
    name="Run 42 - ATP = 0.0312mM", temperature=25, temperature_unit="C",
    ph=7.8, global_time_unit="mins"
)

# Add initial concentrations for reactants
measurement_42.addData(reactant_id=PpanSH_id, init_conc=0.0312, unit="mM")
measurement_42.addData(reactant_id=ATP_id, init_conc=0.0312, unit="mM")
measurement_42.addData(protein_id=PPAT_id, init_conc=0.1, unit="mg/ml")

# Convert time and data columns to lists before passing to Replicate
repeat1 = Replicate(
    id="repeat1 for ATP =0.0312 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df1['Time_min'].tolist(),
    data=df1['F01'].tolist()
)

repeat2 = Replicate(
    id="repeat2 for ATP = 0.0312 mM @ 0.0312 mM PpanSH",
    species_id=ATP_id,
    data_unit="mM",
    time_unit="min",
    time=df2['Time_min'].tolist(),
    data=df2['F03'].tolist()
)



# Add replicates to measurement
measurement_42.addReplicates(repeat1, enzmldoc)
measurement_42.addReplicates(repeat2, enzmldoc)

# Add the measurement to the EnzymeML document
meas_42_id = enzmldoc.addMeasurement(measurement_42)

print("Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.0312 mM ")

Measurement created for two repeats at ATP = 0.0312 mM, PpanSH = 0.0312 mM 


## Add Kinetic Model


In [47]:
# Create models py parsing equations for PPAT

kinmod_PPAT = KineticModel.fromEquation("IMM_PPAT", 
                             "(vf*s0*s1)/((s0+K_ppansh)*(s1+K_atp))")

In [48]:
# Configure parameters for models

kinmod_PPAT._vf.update(value=2.705, unit="mM / min", lower=1e-6)
kinmod_PPAT._K_atp.update(value=0.034, unit="mM", lower=1e-6)
kinmod_PPAT._K_ppansh.update(value=0.0669, unit="mM", lower=1e-6)


In [49]:
# Add models to each reaction

reaction1.model = kinmod_PPAT

In [71]:
# Finally, write to EnzymeML
enzmldoc.toFile(".", "Enzyme ML doc")


Archive was written to .\Enzyme_ML_doc.omex



In [72]:
import numpy as np
from pyenzyme import EnzymeMLDocument

In [46]:
enzmldoc

Phosphopantetheine adenylyltransferase (PPAT)
>>> Reactants
	ID: s0 	 Name: 4'-Phosphopantetheine
	ID: s1 	 Name: Adenosine triphosphate
	ID: s2 	 Name: Inorganic Pyrophospahte
>>> Proteins
	ID: p0 	 Name: PPAT
>>> Complexes
>>> Reactions
	ID: r0 	 Name: Phosphopantetheine adenylyltransferase